In [ ]:
import numpy as np
import hls4ml
import plotting
import tensorflow as tf
import h5py
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from create_dataset import create_dataset
from models import conv_ae, garnet_ae


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# ----------------------------------
# Train - WIP!
# ----------------------------------

dataset_loc = "./dataset"


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# GPU config
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

# load dataset from the regular dataset
h5f = h5py.File(dataset_loc, "r")
X_train = h5f["X_train"][()]
X_test = h5f["X_test"][()]
Y_train = h5f["Y_train"][()]
Y_test = h5f["Y_test"][()]

# define callbacks
callbacks=[
    EarlyStopping(monitor='val_loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
    # tfmot.sparsity.keras.UpdatePruningStep() # pruning
]

# get the CNN autoencoder
model, encoder = conv_ae(quant_size=0, pruning=False)

# begin training
batch_size = 1024
n_epochs = 20

hist = model.fit(
        x=X_train,
        y=Y_train,
        epochs=n_epochs,
        batch_size=batch_size,
        verbose=2,
        validation_split=0.2,
        callbacks=callbacks)

pred = model.predict(X_test)
encoded = encoder.predict(X_test)

config = hls4ml.utils.config_from_keras_model(model, granularity='model')
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       output_dir='model_1/hls4ml_prj',
                                                       fpga_part='xcu250-figd2104-2L-e')

hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)
hls_model.compile()
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(pred, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_hls, axis=1))))

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(Y_test, pred, le.classes_)
plt.gca().set_prop_cycle(None) # reset the colors
_ = plotting.makeRoc(Y_test, y_hls, le.classes_, linestyle='--')

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], ls='-'),
         Line2D([0], [0], ls='--')]
from matplotlib.legend import Legend
leg = Legend(ax, lines, labels=['keras', 'hls4ml'],
            loc='lower right', frameon=False)
ax.add_artist(leg)